<h1>DATA ANALYSIS OF NEW YORK CITI BIKERS IN SQL </h1>


<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="https://#remove duplicates">Identify and remove duplicates</a></li>
    <li><a href="https://#identify_handle_missing_values">Identify and handle missing values</a>
        <ul>
            <li><a href="https://#identify_missing_values">Identify missing values</a></li>
            <li><a href="https://#deal_missing_values">Deal with missing values</a></li>
            <li><a href="https://#correct_data_format">Correct data format</a></li>
        </ul>
    </li>
    <li><a href="https://#data_standardization">Data standardization</a></li>
    <li><a href="https://#data_normalization">Data normalization (centering/scaling)</a></li>
    <li><a href="https://#binning">Binning</a></li>
    <li><a href="https://#indicator">Indicator variable</a></li>
</ul>

</div>

<hr>


# Install SQLAlchemy

In [1]:
!pip install sqlalchemy

In [2]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


# Install ipython-sql

This is because you might have issues loading ext slq

In [3]:
!pip install ipython-sql

# Import software libraries

In [4]:
import pandas as pd
import sqlalchemy
import sqlite3

In [5]:
%load_ext sql

# Create a SQLite connection

In [6]:
engine = sqlalchemy.create_engine('sqlite:///citibikes.db')

# Connect to the SQLite database

In [7]:
%sql sqlite:///citibikes.db

# Load the dataset as a `DataFrame`

In [8]:
NY_Citi_Bikes_data=pd.read_csv('Datasets/New York Citi Bikes_Raw Data - NYCitiBikes (1).csv')

In [9]:
NY_Citi_Bikes_data

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,01-01-17 00:38,1-1-17 01:03,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,01-01-17 01:47,01-01-17 01:58,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,01-01-17 01:47,01-01-17 01:58,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,01-01-17 01:56,01-01-17 02:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,1-1-17 02:12,01-01-17 02:23,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,31-03-17 21:06,31-03-17 21:10,3186,Grove St PATH,3203,Hamilton Park,24608,Subscriber,1968,53,45-54,244,4,3,Spring,18,Friday
20396,31-03-17 21:13,31-3-17 21:18,3195,Sip Ave,3194,McGinley Square,29219,Subscriber,1956,65,65-74,264,4,3,Spring,18,Friday
20397,31-03-17 21:13,31-3-17 21:18,3195,Sip Ave,3194,McGinley Square,29219,Subscriber,1956,65,65-74,264,4,3,Spring,18,Friday
20398,31-03-17 21:26,31-03-17 21:30,3195,Sip Ave,3201,Dey St,29276,Subscriber,1970,51,45-54,239,4,3,Spring,18,Friday


In [10]:
NY_Citi_Bikes_data.head(3)

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,01-01-17 00:38,1-1-17 01:03,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,01-01-17 01:47,01-01-17 01:58,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,01-01-17 01:47,01-01-17 01:58,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday


# Store the DataFrame in the SQLite database

In [11]:
NY_Citi_Bikes_data.to_sql('NYCitiBikesdata', con=engine, if_exists='replace')

20400

In [12]:
results = %sql SELECT * FROM NYCitiBikesdata;

 * sqlite:///citibikes.db
Done.


conn = sqlite3.connect('citibikes.db')
cursor=conn.cursor()
data=cursor.execute('''SELECT * FROM NYCitiBikesdata''')
print(data.description)

# Identify all duplicated data

In [17]:
#To select a column name with spaces, use the back tick symbol with column name. The symbol is ( ` `). Back tick is displayed in the keyboard below the tilde operator ( ~)

duplicated_data = %sql SELECT * from NYCitiBikesdata GROUP BY `Start Time`, `Stop Time` HAVING COUNT(*) > 1


 * sqlite:///citibikes.db
Done.


In [18]:
#duplicated_data

# Remove the duplicated data

In [19]:
NY_Citi_Bikes_data_wt_duplicates= %sql SELECT DISTINCT * FROM NYCitiBikesdata GROUP BY ;

 * sqlite:///citibikes.db
Done.


In [1]:
#NY_Citi_Bikes_data_wt_duplicates

In [ ]:
NY_Citi_Bikes_data_wt_duplicates=NY_Citi_Bikes_data[~NY_Citi_Bikes_data.duplicated()]

In [ ]:
NY_Citi_Bikes_data_wt_duplicates

# Identify and handle missing values

In the dataset, missing data comes with the question mark "?". We replace "?" with NaN (Not a Number), Python's default missing value marker for reasons of computational speed and convenience. Here we use the function:
.replace(A, B, inplace = True) 
to replace A by B.

In [ ]:
import numpy as np

# replace "?" to NaN
NY_Citi_Bikes_data_wt_duplicates.replace("?", np.nan, inplace = True)


# Evaluating for Missing Data

The missing values are converted by default. We use the following functions to identify these missing values. There are two methods to detect missing data:

.isnull()
.notnull()
The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.

In [ ]:
missing_data =NY_Citi_Bikes_data_wt_duplicates.isnull()
missing_data.head(5)

"True" means the value is a missing value while "False" means the value is not a missing value.

# Count missing values in each column

Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value and "False" means the value is present in the dataset. In the body of the for loop the method ".value_counts()" counts the number of "True" values.

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Based on the summary above, each column has 18450 rows of data and only one of the columns containing missing data:

<ul>
    <li>"End Station Name": 1 missing data</li>
</ul>
To deal with this missing data, we will be deleting the entire row
let's drop all rows that do not have End Station Name data

In [ ]:
# Drop whole row with NaN in "End Station Name" column
NY_Citi_Bikes_data_wt_duplicates.dropna(subset=["End Station Name"], axis=0, inplace=True)

# reset index, because we droped one row
NY_Citi_Bikes_data_wt_duplicates.reset_index(drop=True, inplace=True)

In [ ]:
NY_Citi_Bikes_data_wt_duplicates

Good! Now, we have a dataset with no duplicates and no missing values.

# Check to see if all data formats are correct

In [ ]:
NY_Citi_Bikes_data_wt_duplicates.dtypes

In [ ]:
NY_Citi_Bikes_data_wt_duplicates.info()

In [ ]:
#NY_Citi_Bikes_data_wt_duplicates['Start Time']=pd.to_datetime(NY_Citi_Bikes_data_wt_duplicates['Start Time'], format='%d-%m-%Y %H:%M')

Remove all commas from the column 'Trip_Duration_in_min'

In [ ]:
#Remove all commas from the column 
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min']=\
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].str.replace(',','')


In [ ]:
#Change the column data type to int64
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min']=\
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].astype('int64')

In [ ]:
NY_Citi_Bikes_data_wt_duplicates.info()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates

# Calculate descriptive statistics for “Trip duration” Column

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].mean()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].median()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].min()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Trip_Duration_in_min'].max()

# Calculate descriptive statistics for “Age” Column

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Age'].mean()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Age'].median()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Age'].min()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates['Age'].max()

In [ ]:
NY_Citi_Bikes_data_wt_duplicates.describe()

There is an extreme value in column 'Trip_Duration_in_min'. We will remove this outlier from the dataset and stop it from messing up our result.
Delete the entire row associated with the outlier

In [ ]:
#Delete the entire row with 'Trip_Duration_in_min' = 6515
NY_Citi_Bikes_data_wt_duplicates =\
NY_Citi_Bikes_data_wt_duplicates.drop(index=[row for row in NY_Citi_Bikes_data_wt_duplicates.index if 6515 == NY_Citi_Bikes_data_wt_duplicates.loc[row, 'Trip_Duration_in_min']])

In [ ]:
NY_Citi_Bikes_data_wt_duplicates

In [ ]:
NY_Citi_Bikes_data_wt_duplicates.describe()